## 11) 워드 임베딩의 평균(Average Word Embedding)

앞서 특정 문장 내의 단어들의 임베딩 벡터들의 평균이 그 문장의 벡터가 될 수 있음을 설명했다. 이번 챕터에서는 임베딩이 잘 된 상황에서는 단어 벡터들의 평균만으로 텍스트 분류를 수행할 수 있음을 보이고, 워드 임베딩의 중요성을 이해해보겠다.

영화 사이트 IMDB 영화 리뷰 데이터는 리뷰 텍스트에 리뷰가 긍정인 경우 1을, 부정인 경우 0을 레이블링한 데이터로 25,000개의 훈련 데이터와 테스트 데이터 25,000개로 구성된 데이터이다. 케라스를 통해서 이 데이터셋을 바로 다운로드할 수 있다.

이 데이터에 대한 더 자세한 설명은 11챕터의 'IDMB 리뷰 감성 분류하기' 실습에서 확인할 수 있다. 여기서는 데이터나 코드에 대한 설명보다는 단어 벡터의 평균으로 얻을 수 있는 성능에 주목하자.



### 1.데이터 로드와 전처리


In [1]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.datasets import imdb

케라스에서는 imdb.data_load()를 통해서 IMDB 리뷰 데이터를 다운로드할 수 있는데, 데이터를 로드할 때 파라미터로 num_words를 사용하면 이 데이터에서 등장 빈도 순위로 몇 번째에 해당하는 단어까지를 사용할 것인지를 의미한다. 만약 vocab_size를 20,000으로 지정할 경우 등장 빈도 순위가 20,000등이 넘는 단어들은 불러올 때, 불용어를 제거하듯 전체 데이터에서 전부 제거 후 로드한다.



In [2]:
vocab_size = 20000

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = vocab_size)

print('훈련용 리뷰 개수 :', len(x_train))
print('테스트용 리뷰 개수 :', len(x_test))

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])


훈련용 리뷰 개수 : 25000
테스트용 리뷰 개수 : 25000


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


이 데이터는 이미 정수 인코딩까지의 전처리가 진행되어져 있다. 그래서 단어 집합을 만들고, 각 단어를 정수로 인코딩하는 과정을 할 필요가 없다. 가령, 첫번째 리뷰를 출력해보자.



In [3]:
print(x_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


첫번째 리뷰를 출력했더니 정수 시퀀스가 출력된다. 첫번째 리뷰의 레이블을 출력해보자.


In [4]:
print(y_train[0])

1


정수 1이 출력되는데, 이는 첫번째 리뷰가 긍정 리뷰임을 의미한다. 각 리뷰의 평균 길이를 계산해보자.



In [5]:
print('훈련용 리뷰의 평균 길이 : {}'.format(np.mean(list(map(len, x_train)), dtype = int)))
print('테스트용 리뷰의 평균 길이 : {}'.format(np.mean(list(map(len, x_test)), dtype = int)))


훈련용 리뷰의 평균 길이 : 238
테스트용 리뷰의 평균 길이 : 230


훈련용 리뷰와 테스트용 리뷰의 평균 길이가 각각 238, 230이다. 평균보다는 큰 수치인 400으로 두 리뷰 데이터를 패딩하겠다.



In [6]:
max_len = 400

x_train = pad_sequences(x_train, maxlen = max_len)
x_test  = pad_sequences(x_test, maxlen = max_len)
print('x_train의 크기(shape) :', x_train.shape)
print('x_test의 크기(shape) :', x_test.shape)


x_train의 크기(shape) : (25000, 400)
x_test의 크기(shape) : (25000, 400)


### 2.모델 설계하기

이제 모델의 입력으로 사용하기 위한 모든 전처리를 마쳤다. 임베딩 벡터를 평균으로 사용하는 모델을 설계해보자. GlobalAveragePooling1D()은 입력으로 들어오는 모든 벡터들의 평균을 구하는 역할을 한다. 다시 말해 Embedding() 다음에 GlobalAveragePooling1D()을 추가하면, 해당 문장의 모든 단어 벡터들의 평균 벡터를 구하게 된다.

이진 분류를 수행해야 하므로 그 후에는 시그모이드 함수를 활성화 함수로 사용하는 뉴런 1개를 배치한다.


In [21]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length = max_len))
model.add(GlobalAveragePooling1D()) # 모든 단어 벡터의 평균을 구한다.
model.add(Dense(1, activation = 'sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 400, 50)           1000000   
_________________________________________________________________
global_average_pooling1d_1 ( (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 1,000,051
Trainable params: 1,000,051
Non-trainable params: 0
_________________________________________________________________


In [22]:
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 4)
mc = ModelCheckpoint('embedding_average_model.h5', monitor = 'val_accuracy', mode = 'max',
                     verbose = 1, save_best_only = True)

훈련 데이터의 20%를 검증 데이터로 사용하고, 총 10회 학습한다.


In [23]:
model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [24]:
model.fit(x_train, y_train,
          batch_size = 32,
          epochs = 10,
          callbacks = [es, mc],
          validation_split = 0.2)


Epoch 1/10
625/625 [==============================] - 9s 14ms/step - loss: 0.6708 - accuracy: 0.6606 - val_loss: 0.5360 - val_accuracy: 0.8074

Epoch 00001: val_accuracy improved from -inf to 0.80740, saving model to embedding_average_model.h5
Epoch 2/10
625/625 [==============================] - 9s 14ms/step - loss: 0.4855 - accuracy: 0.8386 - val_loss: 0.3981 - val_accuracy: 0.8560

Epoch 00002: val_accuracy improved from 0.80740 to 0.85600, saving model to embedding_average_model.h5
Epoch 3/10
625/625 [==============================] - 10s 16ms/step - loss: 0.3518 - accuracy: 0.8836 - val_loss: 0.3353 - val_accuracy: 0.8782

Epoch 00003: val_accuracy improved from 0.85600 to 0.87820, saving model to embedding_average_model.h5
Epoch 4/10
625/625 [==============================] - 9s 14ms/step - loss: 0.2815 - accuracy: 0.9047 - val_loss: 0.3061 - val_accuracy: 0.8838

Epoch 00004: val_accuracy improved from 0.87820 to 0.88380, saving model to embedding_average_model.h5
Epoch 5/10
625

In [25]:
ls

embedding_average_model.h5  sample_data/


학습이 끝난 후에 테스트 데이터에 대해서 평가한다.


In [26]:
loaded_model = load_model('embedding_average_model.h5')
print('\n 테스트 정확도: %.4f' % (loaded_model.evaluate(x_test, y_test)[1]))

782/782 [==============================] - 2s 2ms/step - loss: 0.2880 - accuracy: 0.8860

 테스트 정확도: 0.8860


별 다른 신경망을 추가하지 않고, 단어 벡터의 평균만으로도 88.76%라는 높은 정확도를 얻어낸다.
